<a href="https://colab.research.google.com/github/AthiramolCusat/MacchineLearning/blob/main/VGG%2016.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers
from tensorflow.keras import Model

# Define paths to your dataset
train_path = "/content/drive/MyDrive/archive(16)/training_set"
test_path = "/content/drive/MyDrive/archive(16)/test_set/test_set"

# Set up data augmentation for the training set
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

# Set up data augmentation for the testing set
test_datagen = ImageDataGenerator(rescale=1./255)

# Set up the training set generator
train_generator = train_datagen.flow_from_directory(train_path,
                                                    target_size=(224, 224),
                                                    batch_size=32,
                                                    class_mode='binary')

# Set up the testing set generator
test_generator = test_datagen.flow_from_directory(test_path,
                                                  target_size=(224, 224),
                                                  batch_size=32,
                                                  class_mode='binary')

# Load pre-trained VGG16 model without the top (fully connected) layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the convolutional layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom top layers for binary classification (cat or dog)
x = layers.Flatten()(base_model.output)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(1, activation='sigmoid')(x)

# Create the final model
model = Model(base_model.input, x)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_generator, epochs=2, validation_data=test_generator)

# Evaluate the model on the testing set
accuracy = model.evaluate(test_generator)[1]
print(f"Accuracy on the testing set: {accuracy}")


Found 520 images belonging to 1 classes.
Found 230 images belonging to 2 classes.
Epoch 1/2
17/17 [==============================] - 424s 25s/step - loss: 0.0158 - accuracy: 0.9942 - val_loss: 106.9442 - val_accuracy: 0.4826
Epoch 2/2
8/8 [==============================] - 123s 15s/step - loss: 123.3540 - accuracy: 0.4826
Accuracy on the testing set: 0.48260870575904846


In [2]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the trained model
#model_path = "path/to/your/trained/model.h5"
#model = tf.keras.models.load_model(model_path)

# Path to the new image you want to classify
new_image_path = "/content/drive/MyDrive/archive(16)/test_set/test_set/cats/cat.4159.jpg"

# Load and preprocess the image
img = image.load_img(new_image_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0

# Make predictions
predictions = model.predict(img_array)

# Classify the image
if predictions[0] > 0.5:
    print("Predicted: Dog")
else:
    print("Predicted: Cat")


1/1 [==============================] - 1s 961ms/step
Predicted: Cat


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
